# **elmo1-rhog_nanobody_design**

## **Fig. 3g**

In [ ]:
!pip3 install --upgrade MDAnalysis
# Import MDanalysis
import MDAnalysis as mda
#from MDAnalysis.tests.datafiles import PSF, DCD, DCD2
from MDAnalysis.analysis import gnm
import matplotlib.pyplot as plt
%matplotlib inline
#Installing py3Dmol using pip
!pip install py3Dmol
import py3Dmol
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import glob
from scipy import stats


def rmsd_unalign_align(mobile, ref):

    # RMSD between 1st frame and last frame (without align trajectory)

    mobile.trajectory[-1]  # set mobile trajectory to last frame
    ref.trajectory[0]  # set reference trajectory to first frame

    mobile_ca = mobile.select_atoms('name CA')
    ref_ca = ref.select_atoms('name CA')
    rmsd_without_align = rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition=False)

    # RMSD between 1st frame and last frame (with align trajectory)
    aligner = align.AlignTraj(mobile, ref, select='name CA', in_memory=True).run()

    mobile.trajectory[-1]  # set mobile trajectory to last frame
    ref.trajectory[0]  # set reference trajectory to first frame

    mobile_ca = mobile.select_atoms('name CA')
    ref_ca = ref.select_atoms('name CA')
    rmsd_with_align = rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition=False)

    # Alternatively
    # rms.rmsd(mobile.select_atoms('backbone').positions,  # coordinates to align
    #          ref.select_atoms('backbone').positions,  # reference coordinates
    #          center=True,  # subtract the center of geometry
    #         superposition=True)  # superimpose coordinates

    print("rmsd_without_align =", rmsd_without_align)
    print("rmsd_with_align =", rmsd_with_align)

def rmsd_df(mobile, ref):

    # Here we define two 
    Loop1 = 'backbone and resid 57-69'
    Loop2 = 'backbone and resid 107-138'

    #Here we calculate the RMSD
    R_rmsd = rms.RMSD(mobile,  # universe to align
                ref,  # reference universe or atomgroup
                select='backbone',  # group to superimpose and calculate RMSD
                groupselections=[Loop1, Loop2],  # groups for RMSD
                ref_frame=0)  # frame index of the reference
    R_rmsd.run()

    # Convert to df
    df_rmsd = pd.DataFrame(R_rmsd.results.rmsd,
                          columns=['Frame', 'Time (ns)','Backbone','Loop1','Loop2'])

    return df_rmsd

# get all trajectories
filepath_list = []

for filepath in glob.iglob('./*_repeat*/'):
  filepath_list.append(filepath)

filepath_list.sort()

filepath_list_nano0906 = [ i for i in filepath_list if 'nano-0906' in i ]
filepath_list_nano79 = [ i for i in filepath_list if 'nano-79' in i ]

# tmp
#filepath_list_nano0906 = filepath_list_nano0906[:1]
#filepath_list_nano79 = filepath_list_nano79[:1]

df_rmsd_nano79_cdr2 = {}
df_rmsd_nano0906_cdr2 = {}
df_rmsd_nano79_cdr3 = {}
df_rmsd_nano0906_cdr3 = {}

for filepath_nano79, filepath_nano0906 in zip(filepath_list_nano79, filepath_list_nano0906):

    #Here we create two Universes each containing the same trajectory of the monomeric trajectory, 
    #one called mobile and the other ref which will be used as reference 
    mobile_nano79 = mda.Universe(filepath_nano79 + 'protein_fit.gro', filepath_nano79 + 'protein_fit.xtc')
    ref_nano79 = mda.Universe(filepath_nano79 + 'protein_fit.gro', filepath_nano79 + 'protein_fit.xtc')

    mobile_nano0906 = mda.Universe(filepath_nano0906 + 'protein_fit.gro', filepath_nano0906 + 'protein_fit.xtc')
    ref_nano0906 = mda.Universe(filepath_nano0906 + 'protein_fit.gro', filepath_nano0906 + 'protein_fit.xtc')

    # unaligned vs aligned RMSD
    rmsd_unalign_align(mobile_nano79, ref_nano79)
    rmsd_unalign_align(mobile_nano0906, ref_nano0906)

    # RMSD df
    df_rmsd_nano79 = rmsd_df(mobile_nano79, ref_nano79)
    df_rmsd_nano0906 = rmsd_df(mobile_nano0906, ref_nano0906)

    # append
    repeat_name_nano79 = filepath_nano79.split('/')[1]
    repeat_name_nano0906 = filepath_nano0906.split('/')[1]
    df_rmsd_nano79_cdr2[repeat_name_nano79] = df_rmsd_nano79['Loop1']
    df_rmsd_nano0906_cdr2[repeat_name_nano0906] = df_rmsd_nano0906['Loop1']
    df_rmsd_nano79_cdr3[repeat_name_nano79] = df_rmsd_nano79['Loop2']
    df_rmsd_nano0906_cdr3[repeat_name_nano0906] = df_rmsd_nano0906['Loop2']

# format

def format_rmsd_df(df_dict):

    df = pd.DataFrame(df_dict)
    df['rmsd_mean'] = df.mean(axis=1)
    df['rmsd_min'] = df.min(axis=1)
    df['rmsd_max'] = df.max(axis=1)
    df.insert(0, 'Time (ns)',df_rmsd_nano79['Time (ns)'])
    df.insert(0, 'Frame',df_rmsd_nano79['Frame'])

    return df

df_rmsd_nano79_cdr2 = format_rmsd_df(df_rmsd_nano79_cdr2)
df_rmsd_nano0906_cdr2 = format_rmsd_df(df_rmsd_nano0906_cdr2)
df_rmsd_nano79_cdr3 = format_rmsd_df(df_rmsd_nano79_cdr3)
df_rmsd_nano0906_cdr3 = format_rmsd_df(df_rmsd_nano0906_cdr3)

############
''' CDR2 '''
############

# band graph
fig, ax = plt.subplots(figsize=(4, 2), dpi=1200)

mean_plot = ax.plot(df_rmsd_nano79_cdr2['Frame'], df_rmsd_nano79_cdr2['rmsd_mean'], color='#34C1D6', linewidth=1.5, linestyle = "-", alpha=1.0)
scatter_plot = ax.scatter(df_rmsd_nano79_cdr2['Frame'], df_rmsd_nano79_cdr2['nano-79_apo_repeat1'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr2['Frame'], df_rmsd_nano79_cdr2['nano-79_apo_repeat2'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr2['Frame'], df_rmsd_nano79_cdr2['nano-79_apo_repeat3'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr2['Frame'], df_rmsd_nano79_cdr2['nano-79_apo_repeat4'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr2['Frame'], df_rmsd_nano79_cdr2['nano-79_apo_repeat5'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
min_max_plot = ax.fill_between(df_rmsd_nano79_cdr2['Frame'], df_rmsd_nano79_cdr2['rmsd_min'], df_rmsd_nano79_cdr2['rmsd_max'], lw=0, color='#34C1D6', alpha=0.3)

mean_plot = ax.plot(df_rmsd_nano0906_cdr2['Frame'], df_rmsd_nano0906_cdr2['rmsd_mean'], color='#ff3366', linewidth=1.5, linestyle = "-", alpha=1.0)
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr2['Frame'], df_rmsd_nano0906_cdr2['nano-0906_apo_repeat1'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr2['Frame'], df_rmsd_nano0906_cdr2['nano-0906_apo_repeat2'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr2['Frame'], df_rmsd_nano0906_cdr2['nano-0906_apo_repeat3'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr2['Frame'], df_rmsd_nano0906_cdr2['nano-0906_apo_repeat4'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr2['Frame'], df_rmsd_nano0906_cdr2['nano-0906_apo_repeat5'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
min_max_plot = ax.fill_between(df_rmsd_nano0906_cdr2['Frame'], df_rmsd_nano0906_cdr2['rmsd_min'], df_rmsd_nano0906_cdr2['rmsd_max'], lw=0, color='#ff3366', alpha=0.3)

plt.title('CDR2', size = 12)
plt.xlabel('Frame', size=12)
plt.ylabel('RMSD (Å)', size=12)
ax.yaxis.get_ticklocs(minor=False)
ax.minorticks_on()
#plt.legend()
plt.tight_layout()
plt.show()


############
''' CDR3 '''
############

# band graph
fig, ax = plt.subplots(figsize=(4, 2), dpi=1200)

mean_plot = ax.plot(df_rmsd_nano79_cdr3['Frame'], df_rmsd_nano79_cdr3['rmsd_mean'], color='#34C1D6', linewidth=1.5, linestyle = "-", alpha=1.0)
scatter_plot = ax.scatter(df_rmsd_nano79_cdr3['Frame'], df_rmsd_nano79_cdr3['nano-79_apo_repeat1'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr3['Frame'], df_rmsd_nano79_cdr3['nano-79_apo_repeat2'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr3['Frame'], df_rmsd_nano79_cdr3['nano-79_apo_repeat3'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr3['Frame'], df_rmsd_nano79_cdr3['nano-79_apo_repeat4'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano79_cdr3['Frame'], df_rmsd_nano79_cdr3['nano-79_apo_repeat5'], s=0.5, marker = '.', facecolors='#34C1D6', edgecolors='none')
min_max_plot = ax.fill_between(df_rmsd_nano79_cdr3['Frame'], df_rmsd_nano79_cdr3['rmsd_min'], df_rmsd_nano79_cdr3['rmsd_max'], lw=0, color='#34C1D6', alpha=0.3)

mean_plot = ax.plot(df_rmsd_nano0906_cdr3['Frame'], df_rmsd_nano0906_cdr3['rmsd_mean'], color='#ff3366', linewidth=1.5, linestyle = "-", alpha=1.0)
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr3['Frame'], df_rmsd_nano0906_cdr3['nano-0906_apo_repeat1'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr3['Frame'], df_rmsd_nano0906_cdr3['nano-0906_apo_repeat2'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr3['Frame'], df_rmsd_nano0906_cdr3['nano-0906_apo_repeat3'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr3['Frame'], df_rmsd_nano0906_cdr3['nano-0906_apo_repeat4'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
scatter_plot = ax.scatter(df_rmsd_nano0906_cdr3['Frame'], df_rmsd_nano0906_cdr3['nano-0906_apo_repeat5'], s=0.5, marker = '.', facecolors='#ff3366', edgecolors='none')
min_max_plot = ax.fill_between(df_rmsd_nano0906_cdr3['Frame'], df_rmsd_nano0906_cdr3['rmsd_min'], df_rmsd_nano0906_cdr3['rmsd_max'], lw=0, color='#ff3366', alpha=0.3)

plt.title('CDR3', size = 12)
plt.xlabel('Frame', size=12)
plt.ylabel('RMSD (Å)', size=12)
ax.yaxis.get_ticklocs(minor=False)
ax.minorticks_on()
#plt.legend()
plt.tight_layout()
plt.show()